# Self Healing Glass

In [1]:
from lammps import PyLammps

In [4]:
L = PyLammps()

LAMMPS (3 Mar 2020)
  using 16 OpenMP thread(s) per MPI task
LAMMPS output is captured by PyLammps wrapper


#### SET SIMULATION CONDITIONS

In [5]:
L.command("units         lj")
L.command("atom_style    full")
L.command("dimension     3")
L.command("boundary p p p")
L.command("neighbor 0.8 bin")

L.command("bond_style    harmonic")
L.command("pair_style lj/expand 2.5")

In [6]:
L.command("read_data fracturedGlass.data")

Reading data file ...
  orthogonal box = (14.7781 17.1681 14.0773) to (40.0135 44.1059 37.9255)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  17496 atoms
  reading velocities ...
  17496 velocities
  scanning bonds ...
  2 = max bonds/atom
  reading bonds ...
  17280 bonds
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:   0          0          0         
  special bond factors coul: 0          0          0         
  8 = max # of 1-2 neighbors
  8 = max # of 1-3 neighbors
  16 = max # of 1-4 neighbors
  24 = max # of special neighbors
  special bonds CPU = 0.01003 secs
  read_data CPU = 0.0723929 secs


#### POTENTIALS

In [7]:
L.command("bond_coeff    * 2500.0 0.9")
L.command("special_bonds fene")
L.command("pair_coeff * * 1.0 1.0 0.0 2.5")

Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:   0          1          1         
  special bond factors coul: 0          1          1         
  8 = max # of 1-2 neighbors
  24 = max # of special neighbors
  special bonds CPU = 0.00364804 secs


#### VARIABLES DEFINITION

In [8]:
L.command("variable Temp equal 0.1")
L.command("variable dt equal 0.005")
L.command("variable Tdamping equal 100*${dt}")

#### THERMO-STYLE + INTEGRATION

In [9]:
L.command("thermo  200")
L.command("thermo_style    custom step temp ebond epair ke etotal enthalpy vol press")
L.command("velocity     all create ${Temp} 12398412")
L.command("timestep     ${dt}")

#### PRODUCTION

In [10]:
L.command("fix nvtProd all nvt temp ${Temp} ${Temp} ${Tdamping}")
L.command("dump mydump all custom 200 Conf.dat id type xu yu zu vx vy vz")
L.command("dump_modify mydump sort id")

L.command("run 3000")

L.command("undump mydump")
L.command("unfix nvtProd")
L.command("reset_timestep 0")

Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 3.3
  ghost atom cutoff = 3.3
  binsize = 1.65, bins = 16 17 15
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair lj/expand, perpetual
      attributes: half, newton on
      pair build: half/bin/newton
      stencil: half/bin/3d/newton
      bin: standard
Setting up Verlet run ...
  Unit style    : lj
  Current step  : 0
  Time step     : 0.005
Per MPI rank memory allocation (min/avg/max) = 61.93 | 61.93 | 61.93 Mbytes
Step Temp E_bond E_pair KinEng TotEng Enthalpy Volume Press 
       0          0.1  0.055004218   -6.7955879   0.14999143   -6.5905923   -6.5290405    16211.683  0.066428066 
     200   0.10045595  0.054743853   -6.7988944   0.15067531   -6.5934752   -7.0279986    16211.683  -0.46894701 
     400  0.099117763  0.053812204   -6.7964465   0.14866815   -6.5939661    -7.314613    16211.683  -0.77773775 
     6

#### OSCILLATIONS

In [11]:
L.command("change_box     all triclinic")

L.command("variable tmp equal lx")
L.command("variable L0 equal ${tmp}")
L.command("variable Amp equal lx*0.15  ") #% of box length
L.command("variable Tp equal 10        ") #1 LJ time = 1ps, Tp=1000 --> 10^12/1000 = 10^9 Hz
L.command("variable runTime equal 2.5*${Tp}/${dt}")

L.command("compute         1 all pressure thermo_temp")
L.command("compute         usual all temp")
L.command("compute         tilt all temp/deform")
L.command("variable        strain equal xy/v_L0")
L.command("fix             strainData all ave/time 1 1 1000 c_1[1] c_1[2] c_1[3] c_1[4] c_1[5] c_1[6] file stress.data")
L.command("thermo_style     custom step temp c_usual density press etotal pxy v_strain")
L.command("thermo_modify   temp tilt")

L.command("fix     nvt1 all nvt/sllod temp ${Temp} ${Temp} ${Tdamping}")
L.command("fix     oscillate all deform 1 xy wiggle ${Amp} ${Tp} units box remap v")

L.command("dump mydumpOsc all custom 200 ConfOsc.dat id type xu yu zu vx vy vz")
L.command("dump_modify mydumpOsc sort id")

L.command("run      ${runTime}")

Changing box ...
  triclinic box = (14.7781 17.1681 14.0773) to (40.0135 44.1059 37.9255) with tilt (0 0 0)
Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 3.3
  ghost atom cutoff = 3.3
  binsize = 1.65, bins = 16 17 15
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair lj/expand, perpetual
      attributes: half, newton on
      pair build: half/bin/newton/tri
      stencil: half/bin/3d/newton/tri
      bin: standard
Setting up Verlet run ...
  Unit style    : lj
  Current step  : 0
  Time step     : 0.005
Per MPI rank memory allocation (min/avg/max) = 62.19 | 62.19 | 62.19 Mbytes
Step Temp c_usual Density Press TotEng Pxy v_strain 
       0  0.099855956  0.099855956    1.0792217   -1.8301411   -6.5909238 -0.0071173044            0 
     200  0.012095224   0.48905231    1.0792217   -2.1762456   -6.8123098   -1.2759951  0.088167788 
     400   0.12433435   0.34150044   